# **과제1 : 챗봇만들기**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기1
    * Vector DB 준비하기
    * Retriever, LLM를 연결하기

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

sk-proj-khdE8lwvatAlm1puwkMET3BlbkFJHou7kbG9u5GKNlpRRRBC


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩

In [72]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
data.head()

* 벡터 데이터베이스
    * Embedding 모델 : text-embedding-ada-002
    * DB 경로 : ./db



In [73]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./db",  # 경로 지정(현 위치에서 db 폴더 생성)
                    embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* 데이터 입력
    * DF to Vector DB

In [74]:
# Chroma 데이터베이스 인스턴스 생성
database = Chroma(persist_directory = "./db", embedding_function = embeddings)

In [91]:
# 데이터프레임의 텍스트 열(시리즈)을 리스트로 변환
questions = data['구분'].tolist()
answers = data['QA'].tolist()

# 리스트 내용을 각각 document로 변환
documents = [Document(page_content=questions, metadata={"answer": answer}) for questions, answer in zip(questions, answers)]

print(documents)

[Document(page_content='모집/선발', metadata={'answer': '최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.'}), Document(page_content='모집/선발', metadata={'answer': '35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)'}), Document(page_content='모집/선발', metadata={'answer': '미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'}), Document(page_content='모집/선발', metadata={'answer': '직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.'}), Document(page_content='모집/선발', metadata={'answer': '아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지

In [92]:
# Insert
database.add_documents(documents)

['77d37df1-098b-41a9-a151-64952a135ad0',
 'a82401fc-c506-4808-be70-31e51dcb47c6',
 'd0fe2917-0274-4ee3-820b-0e098f79c448',
 '21f39eac-746b-4201-a07e-7ff4befcbc20',
 '3010305b-0b39-4a7c-8c65-d6515f6d1dd9',
 '34277b79-2c6f-48d9-8d34-91dbc78897ca',
 '4a78fa82-7ed3-44cd-b574-e10b8aef4183',
 '54539282-53d7-402e-b779-476a8e32acf7',
 'd4921f6f-e7dd-44fa-968c-cb87cdcb8dfd',
 '75051c28-d54a-439c-ab5e-bffa9fee7be9']

* 입력된 데이터 조회

In [93]:
# 문서 조회: 유사도 점수도 함께 조회
query = "에이블스쿨 지원하고싶어"   # 질문할 문장
k = 5  # 유사도 상위 k 개 문서 가져오기

# 유사도 검색 및 점수 조회
result = database.similarity_search_with_score(query, k=k)  # 데이터베이스에서 유사도가 높은 문서를 가져옴

# 결과 출력
print('-' * 50)
for doc, score in result:
    print(f"유사도 점수: {round(score, 5)}, 문서 내용: {doc.page_content}")  # 문서 내용 표시


--------------------------------------------------
유사도 점수: 0.42585, 문서 내용: 오픈소스 그룹 일각에서는 AI 모델에 가드레일(허용 가능한 범위를 두는 일종의 가이드라인 또는 도구)을 두는 것 자체를 반대하기도 한다. 모델에 아무런 제한 없어야 더 나은 성능을 발휘할 것이라 믿는 곳도 있다. 기업은 현재 사용하는 오픈소스 모델이 가드레일에 대해 어떤 방향을 추구하는지조차 잘 모를 수도 있다.
수기스는 “현재 오픈소스 생성형 AI 모델의 안전성을 평가하는 독립적인 기관은 없다”라며 “유럽의 AI 법은 이러한 문서를 일부 요구할 것이지만, 대부분의 조항은 2026년에야 시행될 것”이라고 밝혔다. 또한 수기스는 “나라면 가능한 한 많은 문서를 확보하고, 모델을 테스트 및 평가하고, 회사 내부에 몇 가지 보호 장치를 마련할 것”이라고 밝혔다.
유사도 점수: 0.43228, 문서 내용: 예술가, 작가, 기타 저작권 소유자들이 대형 AI 기업을 상대로 소송을 제기하고 있다. 하지만 오픈소스 모델이 지적재산권을 침해하고, 그 모델을 제품이나 서비스에 도입한 기업들만 큰돈을 벌고 있다면 어떻게 될까? 기업 사용자가 소송을 당할 수도 있을까?
EY의 구아레라는 “잠재적인 위험성은 있다. 법원에서 진행 중인 관련 소송이 어떤 결과를 맞이할지는 아직 아무도 모른다”라며 “데이터세트에 대한 보상이 있어야 하는 사회로 향하고 있을지도 모른다”라고 분석했다. 또한 구아레라는 “대형 기술 업체는 앞으로 늘어날 저작권 관련 위기를 극복하고 필요한 자금을 확보할 수 있는 더 나은 위치에 있다”라고 평가했다.
유사도 점수: 0.44029, 문서 내용: 모집/선발
유사도 점수: 0.44029, 문서 내용: 모집/선발
유사도 점수: 0.44029, 문서 내용: 모집/선발


## **3.RAG+LLM모델**

* 모델 : RetrievalQA
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정

* database를 retriever로 선언하기

In [83]:
# retriever로 선언
retriever = database.as_retriever(search_kwargs={"k": 3})

* 모델 선언

In [87]:
# LLM 모델 선언
chat = ChatOpenAI(model="gpt-3.5-turbo")
result = chat([HumanMessage(content="안녕하세요!")])
print(result.content)

# RetrievalQA 모델 선언
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )
result = qa(query)

print(result["result"])

안녕하세요! 무엇을 도와드릴까요?
좋은 선택이네요! 에이블스쿨은 교육 기관 중 하나로, 지원하고 싶다면 해당 교육 기관의 웹사이트나 채용 공고를 확인하여 지원 절차와 자격 요건 등을 참고하시면 도움이 될 것입니다. 혹은 교육 기관에 문의하여 자세한 정보를 얻는 것도 좋은 방법일 수 있습니다.


* 모델 확인

In [89]:
result

{'query': '에이블스쿨 지원하고싶어',
 'result': '좋은 선택이네요! 에이블스쿨은 교육 기관 중 하나로, 지원하고 싶다면 해당 교육 기관의 웹사이트나 채용 공고를 확인하여 지원 절차와 자격 요건 등을 참고하시면 도움이 될 것입니다. 혹은 교육 기관에 문의하여 자세한 정보를 얻는 것도 좋은 방법일 수 있습니다.',
 'source_documents': [Document(page_content='오픈소스 그룹 일각에서는 AI 모델에 가드레일(허용 가능한 범위를 두는 일종의 가이드라인 또는 도구)을 두는 것 자체를 반대하기도 한다. 모델에 아무런 제한 없어야 더 나은 성능을 발휘할 것이라 믿는 곳도 있다. 기업은 현재 사용하는 오픈소스 모델이 가드레일에 대해 어떤 방향을 추구하는지조차 잘 모를 수도 있다.\n수기스는 “현재 오픈소스 생성형 AI 모델의 안전성을 평가하는 독립적인 기관은 없다”라며 “유럽의 AI 법은 이러한 문서를 일부 요구할 것이지만, 대부분의 조항은 2026년에야 시행될 것”이라고 밝혔다. 또한 수기스는 “나라면 가능한 한 많은 문서를 확보하고, 모델을 테스트 및 평가하고, 회사 내부에 몇 가지 보호 장치를 마련할 것”이라고 밝혔다.'),
  Document(page_content='예술가, 작가, 기타 저작권 소유자들이 대형 AI 기업을 상대로 소송을 제기하고 있다. 하지만 오픈소스 모델이 지적재산권을 침해하고, 그 모델을 제품이나 서비스에 도입한 기업들만 큰돈을 벌고 있다면 어떻게 될까? 기업 사용자가 소송을 당할 수도 있을까?\nEY의 구아레라는 “잠재적인 위험성은 있다. 법원에서 진행 중인 관련 소송이 어떤 결과를 맞이할지는 아직 아무도 모른다”라며 “데이터세트에 대한 보상이 있어야 하는 사회로 향하고 있을지도 모른다”라고 분석했다. 또한 구아레라는 “대형 기술 업체는 앞으로 늘어날 저작권 관련 위기를 극복하고 필요한 자금을 확보할 수 있는 더 나은 위치에 있다”라고 평가했다.'),
  Document(page_content